In [4]:
import os
import pandas as pd
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup, AutoTokenizer, AutoModelForSequenceClassification
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon

In [ ]:

def calculate_bias(male_preds, female_preds):

    bias = np.mean(np.array(male_preds) - np.array(female_preds))
    total_bias = np.mean(biases)
    absolute_bias = np.mean(np.abs(biases))
    return biases, total_bias, absolute_bias



def test_significance(biases):
    '''
    Determine significance of the biases by employing the Wilcoxon Signed-Rank Test
    '''
    _, p_value = wilcoxon(biases, zero_method = "pratt")
    return p_value


def create_bias_table(model, total_biases, absolute_biases, p_values):
    data = {"Model": model,
            "Total Bias": total_biases,
            "Absolute Bias": absolute_biases,
            "p-value": p_values}
    return pd.DataFrame(data)


def plot_bias(biases, models, labels):
    x = range(len(models))
    plt.bar(x, biases, tick_label = models, color = ['blue' if b >= 0 else 'red' for b in biases])
    plt.xlabel('Models')
    plt.ylabel('Bias')
    plt.title('Bias per Model')
    plt.axhline(0, color = 'black', linewidth = 0.5)
    plt.show()


    # Calculate bias
    biases, total_bias, absolute_bias = calculate_bias(predictions['male']['Predictions'], predictions['female']['Predictions'])
    metrics_data["female"]["Total_Bias"] = total_bias
    metrics_data["male"]["Total_Bias"] = total_bias
    metrics_data["female"]["Abs_Bias"] = absolute_bias
    metrics_data["male"]["Abs_Bias"] = absolute_bias

    p_value = test_significance(biases)
    data = create_bias_table(args.model, total_biases, absolute_biases, p_values)
    plot_bias(biases, models, labels)





## Test

In [112]:
def load_data(file_path):

    df = pd.read_csv(file_path)
    df['Sentiment'] = df['Sentiment'].map({'pos': 1, 'neg': 0})
    
    df['text'] = df['text'].astype(str)
    text = df['text'].tolist()
    labels = df['Sentiment'].tolist()

    return text, labels

F_train_text, F_train_labels = load_data("/work/SofieNørboMosegaard#5741/NLP/NLP-exam/data_2/all_female_test.csv")
M_train_text, M_train_labels = load_data("/work/SofieNørboMosegaard#5741/NLP/NLP-exam/data_2/all_male_test.csv")

# make sure its the right format
print(type(F_train_text))  #  <class 'list'>
print(len(F_train_text))  # 814
print(type(F_train_text[0])) # str

<class 'list'>
814
<class 'str'>


In [113]:
# Load  tokenizer

tokenizer = AutoTokenizer.from_pretrained("vesteinn/DanskBERT")

#finetuned_model= "/work/SofieNørboMosegaard#5741/NLP/NLP-exam/finetuned_models/BERT_finetuned_original"
#tokenizer = AutoTokenizer.from_pretrained(finetuned_model)

tokenizer

XLMRobertaTokenizerFast(name_or_path='vesteinn/DanskBERT', vocab_size=50005, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50004: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [114]:
def tokenize_data(test_text, tokenizer):

    input_ids = []
    attention_masks = []

    for text in test_text:
        encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens = True,
        max_length = 512,
        padding = "max_length",
        truncation = True,
        return_attention_mask = True,
        return_tensors = "pt",
    )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim = 0)
    attention_masks = torch.cat(attention_masks, dim = 0)
    
    return input_ids, attention_masks

# Tokenize the female and male datasets
F_input_ids, F_attention_masks = tokenize_data(F_train_text, tokenizer)
M_input_ids, M_attention_masks = tokenize_data(M_train_text, tokenizer)

# Convert labels to tensors
F_labels = torch.tensor(F_train_labels)
M_labels = torch.tensor(M_train_labels)

# Verify shapes
print(F_input_ids.shape)  # [814, 512]
print(F_attention_masks.shape)  # [814, 512]
print(F_labels.shape)  # [814]

torch.Size([814, 512])
torch.Size([814, 512])
torch.Size([814])


In [115]:
# Create dataset
F_dataset = TensorDataset(F_input_ids, F_attention_masks, F_labels)
M_dataset = TensorDataset(M_input_ids, M_attention_masks, M_labels)

batch_size = 16  # 8, 16

F_dataloader = DataLoader(F_dataset, batch_size = batch_size, shuffle = True)
M_dataloader = DataLoader(M_dataset, batch_size = batch_size, shuffle = True)

In [116]:
# Load pretrained or fine-tuned  BERT

pretrained_model = AutoModelForSequenceClassification.from_pretrained("vesteinn/DanskBERT", num_labels = 2)
pretrained_model.to(device)

#finetuned_model = "/work/SofieNørboMosegaard#5741/NLP/NLP-exam/finetuned_models" # BERT_finetuned_original
#finetuned_model = AutoModelForSequenceClassification.from_pretrained(finetuned_model)
#finetuned_model.to(device)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at vesteinn/DanskBERT and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(50005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=7

In [117]:
def test_model(model, dataloader):

    model.eval()

    predictions = []
    true_labels = []

    for batch in dataloader:

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask = b_input_mask)
            logits = outputs.logits

        # Convert logits to predicted class labels
        preds = torch.argmax(logits, dim = 1).cpu().numpy()
        predictions.extend(preds)
        true_labels.extend(b_labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    }

    return metrics, predictions

In [125]:
F_metrics, F_predictions = test_model(pretrained_model, F_dataloader)
F_metrics

{'accuracy': 0.49017199017199015,
 'precision': 0.4887459807073955,
 'recall': 0.37254901960784315,
 'f1_score': 0.4228094575799722}

In [126]:
M_metrics, M_predictions = test_model(pretrained_model, M_dataloader)
M_metrics

{'accuracy': 0.5,
 'precision': 0.5006180469715699,
 'recall': 0.9926470588235294,
 'f1_score': 0.6655710764174199}

In [120]:
def calculate_bias(female_preds, male_preds):

    bias = np.mean(np.array(female_preds) - np.array(male_preds))

    return bias

pretrained_bias = calculate_bias(F_predictions, M_predictions)
print(f"Pretrained model bias (Male - Female): {pretrained_bias}")

Pretrained model bias (Male - Female): -0.6117936117936118


In [147]:
# save results

metrics_female = {'Model': "Pretrained Model (Female Dataset)",
                'Metrics': F_metrics,
                'Bias': pretrained_bias}

metrics_male = {'Model': "Pretrained Model (Male Dataset)",
                'Metrics': M_metrics,
                'Bias': pretrained_bias}

predictions_female = {'Model': "Pretrained Model (Female Dataset)",
                    'Predictions': F_predictions,
                    'True_Labels': F_train_labels}

predictions_male = {'Model': "Pretrained Model (Male Dataset)",
                    'Predictions': M_predictions,
                    'True_Labels': M_train_labels}

# Save the metrics to a pickle file
with open("/work/SofieNørboMosegaard#5741/NLP/NLP-exam/results/model_results.pkl", 'ab') as f:
    pickle.dump(metrics_female, f)
    pickle.dump(metrics_male, f)

# Save the predictions to a pickle file
with open("/work/SofieNørboMosegaard#5741/NLP/NLP-exam/results/predictions.pkl", 'ab') as f:
    pickle.dump(predictions_female, f)
    pickle.dump(predictions_male, f)


In [152]:
with open("/work/SofieNørboMosegaard#5741/NLP/NLP-exam/results/model_results.pkl", 'rb') as f:
    metrics_data = []
    metrics_data.append(pickle.load(f))

print(metrics_data)

with open("/work/SofieNørboMosegaard#5741/NLP/NLP-exam/results/predictions.pkl", 'rb') as f:
    predictions_data = []
    predictions_data.append(pickle.load(f))

print(predictions_data)


[{'Model': 'Pretrained Model (Female Dataset)', 'Metrics': {'accuracy': 0.49017199017199015, 'precision': 0.4887459807073955, 'recall': 0.37254901960784315, 'f1_score': 0.4228094575799722}, 'Bias': np.float64(-0.6117936117936118)}]
[{'Model': 'Pretrained Model (Female Dataset)', 'Predictions': [np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np